##Source : https://github.com/samcw/ResNet18-Pytorch

## SetUp

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


## Defines the ResidualBlock and ResNet classes

In [2]:
# ResidualBlock: Implements the core residual block of ResNet
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        # Main path: two 3x3 convolutions with BatchNorm and ReLU
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        # Shortcut path: matches dimensions if necessary
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )
            
    def forward(self, x):
        # Combine main path and shortcut, followed by ReLU activation
        out = self.left(x)
        out = out + self.shortcut(x)
        out = F.relu(out)
        
        return out

# ResNet: Constructs the full ResNet architecture
class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()  
        self.inchannel = 64
        # Initial convolution layer
        self.conv1 = nn.Sequential(
            # nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),    # For RGB images
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False), # For grayscale images
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        # Stacked residual layers
        self.layer1 = self.make_layer(ResidualBlock, 64, 2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)        
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)     
        # Fully connected layer for classification   
        self.fc = nn.Linear(512, num_classes)
        
    def make_layer(self, block, channels, num_blocks, stride):
        # Creates a layer of residual blocks
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # Forward pass through ResNet
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)  # Global average pooling
        out = out.view(out.size(0), -1)  # Flatten features
        out = self.fc(out)  # Fully connected layer
        return out

In [3]:
# ResNet18: Returns a ResNet model with ResidualBlock as the building block
def ResNet18():
    return ResNet(ResidualBlock)

In [4]:
print(torch.backends.mps.is_available())

True


## DataSet

### Load DataSet 

In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image

# Check device: MPS (GPU support for Apple Silicon) or CPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Define the transformation pipeline for MNIST
transform_train = transforms.Compose([
    transforms.ToTensor(),  # Converts PIL Image or numpy.ndarray to Tensor
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize for MNIST dataset
])

transform_test = transforms.Compose([
    transforms.ToTensor(),  # Converts PIL Image or numpy.ndarray to Tensor
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize for MNIST dataset
])

# Custom Dataset Loading Function to Ensure PIL Image Format
def pil_transform(img):
    # Check if the image is a numpy.ndarray and convert it to PIL Image if necessary
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)
    return img

# Load the MNIST dataset with transformations
trainset = torchvision.datasets.MNIST(root='../data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


In [14]:
import torch.optim as optim
import torch.nn as nn
# from ResNetModel import ResNet18  # Import the ResNet18 model from your ResNetModel.py

# Set hyperparameters
EPOCH = 10
LR = 0.01

# Initialize the model and move it to the device (GPU/CPU)
net = ResNet18().to(device)  # Initialize the model
net.load_state_dict(torch.load('models/resnet18_mnist.pth'))  # Load the saved weights
net.eval()  # Set the model to evaluation mode

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

# Training loop
for epoch in range(EPOCH):
    print('\nEpoch: %d' % (epoch + 1))
    net.train()  # Set the model to training mode
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    for i, data in enumerate(trainloader, 0):
        # Prepare dataset
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device (MPS/CPU)

        optimizer.zero_grad()  # Zero the parameter gradients

        # Forward & backward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()  # Backpropagation
        optimizer.step()  # Optimize
        
        # Print loss and accuracy per batch
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted labels
        total += labels.size(0)  # Total number of labels
        correct += (predicted == labels).sum().item()  # Compare predicted with true labels
        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% ' 
              % (epoch + 1, (i + 1 + epoch * len(trainloader)), sum_loss / (i + 1), 100. * correct / total))
    
    # Evaluate accuracy with test dataset at the end of each epoch
    print('Waiting for Testing...')
    with torch.no_grad():  # No need to track gradients during testing
        correct = 0
        total = 0
        for data in testloader:
            net.eval()  # Set the model to evaluation mode
            images, labels = data
            images, labels = images.to(device), labels.to(device)  # Move to device (MPS/CPU)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)  # Get the predicted labels
            total += labels.size(0)  # Total number of labels
            correct += (predicted == labels).sum().item()  # Compare predicted with true labels
        print('Test accuracy: %.3f%%' % (100 * correct / total))

print('Training finished. Total epochs: %d' % EPOCH)



Epoch: 1


/var/folders/by/y1pky8h91xz9xqb13jkbvs2w0000gn/T/ipykernel_69189/1294096073.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('models/resne

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/miniconda3/envs/resnet18_env/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/opt/miniconda3/envs/resnet18_env/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/miniconda3/envs/resnet18_env/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/miniconda3/envs/resnet18_env/lib/python3.9/site-packages/torchvision/datasets/mnist.py", line 146, in __getitem__
    img = self.transform(img)
  File "/opt/miniconda3/envs/resnet18_env/lib/python3.9/site-packages/torchvision/transforms/transforms.py", line 95, in __call__
    img = t(img)
  File "/op

In [10]:
# Use ResNet18 with MINST dataset
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
# from ResNetModel import ResNet18  # Assuming ResNet18 is defined in a separate file

# Check device: MPS (GPU support for Apple Silicon) or CPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Set hyperparameters
EPOCH = 10
BATCH_SIZE = 128
LR = 0.01

# Transformations for MNIST dataset
transform_train = transforms.Compose([
    transforms.ToTensor(),  # Converts PIL Image or ndarray to tensor
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize for MNIST dataset
])

transform_test = transforms.Compose([
    transforms.ToTensor(),  # Converts PIL Image or ndarray to tensor
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize for MNIST dataset
])

# Load MNIST dataset with the proper transformations
trainset = torchvision.datasets.MNIST(root='../data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


# Define classes (digits 0-9 for MNIST)
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

# Initialize the model and move it to the device (GPU/CPU)
net = ResNet18().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)


In [11]:
img, label = trainset[0]  # Access the first image in the dataset
print(type(img))  # This should print <class 'PIL.Image.Image'>


TypeError: expected np.ndarray (got numpy.ndarray)

In [12]:
# Inside the training loop or a simple test
img, label = trainset[0]  # Access the first image
print(f"Image type before transformation: {type(img)}")  # Should print PIL.Image.Image

# Then, apply the transformation
img_tensor = transform_train[0](img)  # Apply ToTensor() separately for debugging
print(f"Image type after transformation: {type(img_tensor)}")  # Should print <class 'torch.Tensor'>


TypeError: expected np.ndarray (got numpy.ndarray)

### Training Model

In [8]:
# Train
pre_epoch = 0  # Start training from epoch 0
for epoch in range(pre_epoch, EPOCH):
    print('\nEpoch: %d' % (epoch + 1))
    net.train()  # Set the model to training mode
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    for i, data in enumerate(trainloader, 0):
        # Prepare dataset
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device (MPS/CPU)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward & backward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()  # Backpropagation
        optimizer.step()  # Optimize
        
        # Print loss and accuracy per batch
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted labels
        total += labels.size(0)  # Total number of labels
        correct += predicted.eq(labels.data).cpu().sum()  # Compare predicted with true labels
        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% ' 
              % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
        
    # Evaluate accuracy with test dataset at the end of each epoch
    print('Waiting for Testing...')
    with torch.no_grad():  # No need to track gradients during testing
        correct = 0
        total = 0
        for data in testloader:
            net.eval()  # Set the model to evaluation mode
            images, labels = data
            images, labels = images.to(device), labels.to(device)  # Move to device (MPS/CPU)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)  # Get the predicted labels
            total += labels.size(0)  # Total number of labels
            correct += (predicted == labels).sum().item()  # Compare predicted with true labels
        print('Test accuracy: %.3f%%' % (100 * correct / total))

print('Training finished. Total epochs: %d' % EPOCH)



Epoch: 1


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/miniconda3/envs/resnet18_env/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/opt/miniconda3/envs/resnet18_env/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/miniconda3/envs/resnet18_env/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/miniconda3/envs/resnet18_env/lib/python3.9/site-packages/torchvision/datasets/mnist.py", line 146, in __getitem__
    img = self.transform(img)
  File "/opt/miniconda3/envs/resnet18_env/lib/python3.9/site-packages/torchvision/transforms/transforms.py", line 95, in __call__
    img = t(img)
  File "/op

## Save and load the model

In [8]:
# Save model in models/ directory
torch.save(net.state_dict(), 'models/resnet18_mnist.pth')

NameError: name 'torch' is not defined

In [9]:
# Initialize the model
net = ResNet18().to(device)

# Load the saved weights
net.load_state_dict(torch.load('models/resnet18_mnist.pth'))

# Set the model to evaluation mode (important for inference)
net.eval()

/var/folders/by/y1pky8h91xz9xqb13jkbvs2w0000gn/T/ipykernel_69189/3941642047.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('models/resnet

ResNet(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer1): Sequential(
    (0): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inp